# TP : Classification de critiques IMDb avec PyTorch

## Objectifs

- **Manipuler le dataset IMDb** en utilisant la librairie [datasets](https://huggingface.co/docs/datasets).
- **Prétraiter le texte** : tokenisation, construction d’un vocabulaire, conversion des textes en séquences d’indices et mise à la même longueur (padding).
- **Construire un modèle simple** en PyTorch composé d’une couche d’**embedding** et d’une couche dense.
- **Entraîner et évaluer le modèle** sur un problème de classification binaire (critique positive / critique négative).

## Contexte

Nous souhaitons construire un modèle de classification de critiques de films. Pour cela, nous allons :
- Charger et découper le dataset IMDb.
- Préparer les données pour les faire passer dans un modèle PyTorch.
- Construire un réseau de neurones simple qui se compose d’une couche d’embedding (pour transformer chaque mot en vecteur) et d’une couche dense (pour réaliser la classification).
- Entraîner le modèle et évaluer ses performances.

---

## Questions

- **Simplicité du modèle :** Ce modèle ne prend en compte que l’information globale par une moyenne des embeddings. Quelles seraient les limites de cette approche ?
- **Prétraitement du texte :** Quels outils (par exemple, spaCy, nltk) pourraient être utilisés pour améliorer la tokenisation et la gestion du vocabulaire ?
- **Améliorations possibles :** Proposez des idées afin d'améliorez ce modèle, ajouter des layers ou autre modification

In [ ]:
!pip install -q -U torch datasets numpy torchtext

In [ ]:
from datasets import load_dataset

# Chargement du dataset "imdb" depuis HuggingFace
raw_dataset = load_dataset("imdb", split="train")

# Split stratifié 50/50
dataset = raw_dataset.train_test_split(stratify_by_column="label", test_size=0.5, seed=42)
train_dataset_raw = dataset["train"]
test_dataset_raw = dataset["test"]

print("Nombre d'échantillons dans le train set :", len(train_dataset_raw))
print("Nombre d'échantillons dans le test set  :", len(test_dataset_raw))

In [ ]:
print("Exemple de review :", train_dataset_raw[0]["text"][:300], "...")
print("Label (0 = négatif, 1 = positif) :", train_dataset_raw[0]["label"])

## Prétraitement des données avec torchtext

Pour préparer les textes :
- **Tokenisation** : nous utiliserons le [tokenizer](https://huggingface.co/docs/transformers/en/main_classes/tokenizer) de Hugging Face
- **Construction du vocabulaire** : à partir des tokens du jeu d’entraînement, nous alons créez un dictionnaire associant chaque mot à un identifiant numérique. Pensez à réserver un index pour les tokens inconnus (`<UNK>`) et pour le padding (`<PAD>`).
- **Transformation** : convertissez chaque critique en une séquence d’indices.
- **Padding / Troncature** : pour obtenir des séquences de taille fixe (par exemple, 256 tokens).

In [ ]:
from tokenizers.models import WordLevel
from tokenizers import Tokenizer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import WordLevelTrainer

# Paramètres de prétraitement
MAX_SEQ_LEN = 256  # longueur maximale de la séquence
MIN_FREQ = 2  # fréquence minimale pour intégrer un mot dans le vocabulaire

# Création d'un tokenizer WordLevel
tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()

# Définition des tokens spéciaux
special_tokens = ["[UNK]", "[PAD]"]

# Entraîneur qui prendra en compte uniquement les tokens apparaissant au moins MIN_FREQ fois
trainer = WordLevelTrainer(special_tokens=special_tokens, min_frequency=MIN_FREQ)

# Récupération des textes d'entraînement
texts = [example["text"] for example in raw_dataset]

# Entraînement du tokenizer sur ces textes
tokenizer.train_from_iterator(texts, trainer=trainer)

# Activation de la troncature et du padding pour obtenir des séquences de longueur fixe
tokenizer.enable_truncation(max_length=MAX_SEQ_LEN)
tokenizer.enable_padding(length=MAX_SEQ_LEN,
                         pad_id=tokenizer.token_to_id("[PAD]"),
                         pad_token="[PAD]")

# Affichage de la taille du vocabulaire
vocab_size = len(tokenizer.get_vocab())
print(f"Taille du vocabulaire: {vocab_size}")

In [ ]:
# ont test le tokenizer pour voir les différents tokens
print(tokenizer.encode("Hello worl !").tokens[:10])
print(tokenizer.encode("Hello worl !").ids[:10])

### Création du Dataset PyTorch

In [ ]:
import datasets
from torch.utils.data import DataLoader, Dataset
import torch


class IMDBDataset(Dataset):
    def __init__(self, raw_dataset: datasets.arrow_dataset.Dataset, tokenizer: Tokenizer):
        self.raw_dataset = raw_dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.raw_dataset)

    def __getitem__(self, idx):
        text = self.raw_dataset[idx]["text"]
        label = self.raw_dataset[idx]["label"]
        encoded = self.tokenizer.encode(text)
        return torch.tensor(encoded.ids, dtype=torch.long), torch.tensor(label, dtype=torch.long)


# Création des datasets PyTorch
train_dataset = IMDBDataset(train_dataset_raw, tokenizer)
test_dataset = IMDBDataset(test_dataset_raw, tokenizer)

# DataLoader
BATCH_SIZE = 64
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

### Entraînement et évaluation du modèle

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    # Passer le modèle en mode apprentissage (entraînement)
    # Cette étape permet d'activer certains mécanismes spécifiques à l'entraînement,
    # comme le dropout ou la normalisation par batch.
    model.train()

    # Initialisation des accumulateurs pour la perte totale,
    # le nombre de prédictions correctes et le total des échantillons.
    epoch_loss = 0
    correct = 0
    total = 0

    # Boucle principale traitant chaque lot (batch) d'échantillons dans le DataLoader
    for sequences, labels in loader:
        # Charger les données et leurs étiquettes associées sur le même appareil
        # (GPU ou CPU) pour calculs.
        sequences, labels = sequences.to(device), labels.to(device)

        # Réinitialisation des gradients cumulés de l'optimiseur.
        # Cela empêche d'accumuler des gradients des itérations précédentes.
        optimizer.zero_grad()

        # Passage avant (forward pass) : le modèle génère des prédictions à partir des séquences.
        outputs = model(sequences)

        # Calcul de la perte (erreur) entre les prédictions du modèle et les étiquettes vraies.
        loss = criterion(outputs, labels)

        # Passage arrière (backward pass) : calcul des gradients pour les poids du modèle.
        loss.backward()

        # Mise à jour des poids du modèle avec l'optimiseur selon le gradient calculé.
        optimizer.step()

        # Accumuler la perte totale pour cette époque.
        # La perte est pondérée par la taille du lot actuel pour obtenir une moyenne correcte plus tard.
        epoch_loss += loss.item() * sequences.size(0)

        # Convertir les prédictions du modèle en étiquettes de classe en sélectionnant
        # l'indice de la classe avec la probabilité la plus élevée.
        preds = outputs.argmax(dim=1)

        # Ajouter le nombre de prédictions correctes pour ce lot.
        # Cela compare les étiquettes prédites avec les étiquettes réelles.
        correct += (preds == labels).sum().item()

        # Mettre à jour le nombre total d'échantillons traités.
        total += labels.size(0)

    # Retourner la perte moyenne et la précision pour cette époque.
    # La perte moyenne est la perte totale divisée par le total d'échantillons.
    # La précision est le ratio des prédictions correctes sur toutes les données.
    return epoch_loss / total, correct / total


def evaluate(model, loader, criterion, device):
    """
    Evaluation  et similaire au train mais il n'y a pas de backward pass, nous utilisons un dataset de test
    """
    # Passer le modèle en mode évaluation. Pendant cette phase, certaines couches,
    # comme le dropout ou la normalisation par batch, sont désactivées.
    model.eval()

    # Initialisation des accumulateurs pour les métriques d'évaluation.
    epoch_loss = 0
    correct = 0
    total = 0

    # Empêcher la mise à jour des gradients (désactivation de autograd).
    # Cela permet d'économiser de la mémoire et d'accélérer l'exécution.
    with torch.no_grad():
        # Boucle principale traitant chaque lot (batch) d'échantillons dans le DataLoader
        for sequences, labels in loader:
            # Charger les données et leurs étiquettes sur le même appareil
            # (GPU ou CPU) pour calculs.
            sequences, labels = sequences.to(device), labels.to(device)

            # Passage avant : le modèle prédit les étiquettes à partir des séquences.
            outputs = model(sequences)

            # Calcul de la perte entre les prédictions et les étiquettes réelles.
            loss = criterion(outputs, labels)

            # Accumuler la perte totale pour cette époque, pondérée par la taille du lot actuel.
            epoch_loss += loss.item() * sequences.size(0)

            # Convertir les prédictions du modèle en indices correspondants à la classe
            # avec la probabilité maximale.
            preds = outputs.argmax(dim=1)

            # Ajouter le nombre de prédictions correctes pour ce lot.
            # Cela compare les étiquettes prédites avec les étiquettes réelles.
            correct += (preds == labels).sum().item()

            # Mettre à jour le nombre total d'échantillons traités.
            total += labels.size(0)

    # Retourner la perte moyenne et la précision pour cette phase d'évaluation.
    return epoch_loss / total, correct / total

### Definitions du model

In [ ]:
import torch.nn as nn

class SimpleClassifier(nn.Module):
    def __init__(self, vocab_size: int, embed_dim: int, num_class: int, pad_idx: int):
        super().__init__()
        # TODO
        pass

    def forward(self, x):
        # TODO
        pass

### Train loop

In [ ]:
from torch import optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device : {device}")
EMBED_DIM = 50
NUM_CLASS = 2  # IMDb est un dataset binaire (0: négatif, 1: positif)
pad_idx = tokenizer.token_to_id("[PAD]")
model = SimpleClassifier(vocab_size, EMBED_DIM, NUM_CLASS, pad_idx)
model.to(device)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
NUM_EPOCHS = 5

Test de votre model avent de lancer un train pour debug les potentielles erreur de dimensions

In [ ]:
dummy_input = torch.tensor([tokenizer.encode("Hello world !").ids]).to(device)
print(model(dummy_input))

In [ ]:
for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch + 1}/{NUM_EPOCHS} | "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc * 100:.2f}% | "
          f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc * 100:.2f}%")